# Query news related to flooding

In [1]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [18]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureSet, GeoAccessor
from arcgis.widgets import MapView
from datetime import datetime, date, timedelta
from geodisasters.services import aggregate, hotspots, query
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat

In [19]:
def querying_locations(client: GeoRapidClient, from_date: date, to_date: date) -> FeatureSet:
    """
    Queries the news related to natural disasters
    and returns the location as a feature set.
    
    :param client: The client instance to use for this query.
    :param from_date: Defines the start of the date range.
    :param to_date: Defines the end of the date range.
    
    :return: The locations related to natural disasters.
    """
    featureset_dict = query(client, from_date, to_date, OutFormat.ESRI)
    featureset = FeatureSet.from_dict(featureset_dict)
    return featureset

def plot_features(gis: GIS, featureset: FeatureSet) -> MapView:
    """
    Creates and returns a simple map view with the plotted geospatial features.

    :param gis: The GIS instance to use for the map view.
    :param featureset: The hotspot locations related to natural disasters.
    
    :return: A map view showing the hotspot locations.
    """
    map_view = gis.map('Europe')
    featureset.sdf.spatial.plot(map_view,
                                renderer_type='c', 
                                method='esriClassifyNaturalBreaks',
                                class_count=5, 
                                col='count', 
                                cmap='YlOrRd',
                                alpha=0.35)
    return map_view

def plot_geo(gis: GIS, sdf: GeoAccessor) -> MapView:
    """
    Creates and returns a simple map view with the plotted geospatial features.

    :param gis: The GIS instance to use for the map view.
    :param sdf: The locations as a spatially enabled dataframe.
    
    :return: A map view showing the hotspot locations.
    """
    map_view = gis.map('Europe')
    sdf.spatial.plot(map_view,
                     renderer_type='c', 
                     method='esriClassifyNaturalBreaks',
                     class_count=5, 
                     col='count', 
                     cmap='YlOrRd',
                     alpha=0.35)
    return map_view

## Authenticate using a Rapid API key
The `host` parameter must target the specific host like "geodisasters.p.rapidapi.com". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

In [7]:
client = EnvironmentClientFactory.create_client_with_host('geodisasters.p.rapidapi.com')

## Connect to ArcGIS Online as an anonymous user

In [8]:
gis = GIS()

## Query the news related to natural disasters
Filter only the news related to flooding, and restrict the date range.

In [12]:
from_date = date(2024, 8, 14)
to_date = date(2024, 8, 17)
locations_fset = querying_locations(client, from_date, to_date)
locations_sdf = locations_fset.sdf

In [15]:
locations_sdf[locations_sdf['theme'] == 'FLOOD'].sort_values(by=['count'], ascending=False)

,count,theme,location,from,to,OBJECTID,SHAPE
0,175,FLOOD,"Karlsruhe, Baden-WüBerg, Germany",2024-08-14,2024-08-17,1,"{""x"": 8.3858, ""y"": 49.0047, ""spatialReference""..."
3,135,FLOOD,"Boston, Massachusetts, United States",2024-08-14,2024-08-17,4,"{""x"": -71.0598, ""y"": 42.3584, ""spatialReferenc..."
4,127,FLOOD,"Breckland, Norfolk, United Kingdom",2024-08-14,2024-08-17,5,"{""x"": 0.6667, ""y"": 52.5, ""spatialReference"": {..."
7,110,FLOOD,"Beijing, Beijing, China",2024-08-14,2024-08-17,8,"{""x"": 116.388, ""y"": 39.9289, ""spatialReference..."
8,109,FLOOD,"Saalbach, ThüN, Germany",2024-08-14,2024-08-17,9,"{""x"": 11.75, ""y"": 50.4, ""spatialReference"": {""..."
27,54,FLOOD,"Sarajevo, Federation Of Bosnia And Herzegovina...",2024-08-14,2024-08-17,28,"{""x"": 18.3833, ""y"": 43.85, ""spatialReference"":..."
33,50,FLOOD,"Ussuri River, China (General), China",2024-08-14,2024-08-17,34,"{""x"": 135.078, ""y"": 48.4517, ""spatialReference..."
35,42,FLOOD,"Jilin, Jilin, China",2024-08-14,2024-08-17,36,"{""x"": 126.56, ""y"": 43.8508, ""spatialReference""..."


In [20]:
plot_geo(gis, locations_sdf)

MapView(layout=Layout(height='400px', width='100%'))